In [1]:
import pandas as pd
import os

In [2]:
filesDir = "ipl_csv2/"

In [3]:
# Get the list of names of the CSV files containing corresponding match's data
files = os.listdir(filesDir)
files = list(map(lambda x: filesDir + x, list(filter(lambda x: x.split('.')[0].isdigit(), files))))

In [4]:
columns = ["match_id", "venue", "innings", "batting_team", "bowling_team", "runs"]
# files

In [5]:
# len(files)

In [6]:
# Declare a dictionary to map team names to their respective regions
teams = {"Mumbai Indians": "Mumbai",
         "Kolkata Knight Riders": "Kolkata",
         "Chennai Super Kings": "Chennai",
         "Kings XI Punjab": "Punjab",
         "Punjab Kings": "Punjab",
         "Rising Pune Supergiant": "Pune",
         "Rising Pune Supergiants": "Pune",
         "Pune Warriors": "Pune",
         "Delhi Daredevils": "Delhi",
         "Delhi Capitals": "Delhi",
         "Gujarat Lions": "Gujarat",
         "Deccan Chargers": "Hyderabad",
         "Sunrisers Hyderabad": "Hyderabad",
         "Royal Challengers Bangalore": "Bangalore",
         "Rajasthan Royals": "Rajasthan",
         "Kochi Tuskers Kerala": "Kerala"}

In [7]:
# Declare a function which will process each CSV file/dataset to obtain the required data for both the innings
# of the corresponding match
# and return the final dataset
def getSixOversData():
    main_df = pd.DataFrame(columns = columns + ["balls"])
    for file in files:
        df = pd.read_csv(file)
        df["runs"] = df["runs_off_bat"] + df["extras"]
        df = df[columns + ["ball"]]
        
        innings1 = getInningsData(df, 1)
        innings2 = getInningsData(df, 2)
        main_df = main_df.append(innings1, ignore_index = True)
        main_df = main_df.append(innings2, ignore_index = True)
        
    return main_df

In [8]:
# The following function takes the match dataset and the innings number
# and returns a dataframe of one row with the required summary data for that innings
def getInningsData(df, innings):
#     print(df.dtypes)
#     print(df["innings"] == innings)
    innings_df = df.loc[df["innings"] == innings]
#     print(innings_df.empty)
    if innings_df.empty:
        return None
    else:
        inningsData = innings_df.loc[innings_df.index[0], innings_df.columns[:-2]].values.tolist()
        inningsData[1] = inningsData[1].split(',')[0].replace('.', ' ').strip()
        inningsData[3] = teams[inningsData[3]]
        inningsData[4] = teams[inningsData[4]]
#         print(inningsData)
#         print(type(df.loc[df["ball"] < 6.1, "runs"].sum()))
        totalRuns = innings_df.loc[innings_df["ball"] < 6.1, "runs"].sum()
        totalBalls = innings_df.loc[innings_df["ball"] < 6.1, "ball"].count()
        inningsData += [totalRuns, totalBalls]
#         print(inningsData)
        return pd.DataFrame([inningsData], columns = columns + ["balls"])

In [9]:
# Create the required summary dataset for creating our model
final_df = getSixOversData()

In [10]:
# print(final_df["venue"].values.tolist())

In [12]:
# Create a CSV file for the final dataframe
final_df.to_csv("six_overs_data.csv", index = False)